In [1]:
import numpy as np
import polyscope as ps

from chromalab.drawhelper import draw_vertex_defined_square

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

### Polyscope Settings for Even-Odd Rendering by Irene

In [3]:
ps.init()
ps.set_ground_plane_mode("none")

# FPS for a full frame (one even and one odd frame)
ps.set_max_fps(30)

# May need to adjust this number in the range of (0, 100) to hit target FPS on a specific devide
# If the frame rate is lower than the target, decrease the target_sleep value
ps.set_target_sleep(85)

# Set always redraw to True (or else polyscope tries to optimize and won't draw even-odd correctly)
ps.set_always_redraw(True)

# Some debug options; also available in the Even-Odd gui panel
ps.set_draw_even_frame_first(True)
ps.set_black_out_even_frames(False)
ps.set_black_out_even_frames(False)

# Enable the Even-Odd gui panel
ps.set_build_even_odd_gui_panel(True)

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 83.1


In [4]:
im_h, im_w = 912, 1140
ps.set_window_resizable(True)
ps.set_window_size(im_w, im_h)

In [5]:
# construct camera parameters
intrinsics = ps.CameraIntrinsics(fov_vertical_deg=45, aspect=im_w/im_h)
extrinsics = ps.CameraExtrinsics(root=(0, 0, 1), look_dir=(0., 0., -1), up_dir=(0.,1.,0.))
params = ps.CameraParameters(intrinsics, extrinsics)

# set the viewport view to those parameters
ps.set_view_camera_parameters(params)

ps.set_navigation_style("planar")
ps.set_view_projection_mode("orthographic")

In [6]:
ps.set_automatically_compute_scene_extents(False)

ray_corners = ps.get_view_camera_parameters().generate_camera_ray_corners()
image_extent_at_z0 = (ray_corners * -1/ray_corners[0, 2])[:, :2] # result of setting up equation start + direction * t = [x, y, 0], zero-plane are objects

lower_left = image_extent_at_z0[2]
height = image_extent_at_z0[0][1] - image_extent_at_z0[2][1]
width = image_extent_at_z0[1][0] - image_extent_at_z0[0][0]

ps.set_length_scale(0.5)
low = np.array((lower_left[0],lower_left[1], -1.)) 
high = np.array((lower_left[0] + width, lower_left[1] + height, 1)) 
ps.set_bounding_box(low, high)

In [7]:
# edges for view debugging
for i, coords in enumerate(image_extent_at_z0):
    draw_vertex_defined_square(coords[0], coords[1], width/8, np.zeros((4, 3)), i)

In [8]:
border = width / 10
side = width / 4
lower_left_subimage = lower_left + np.array([border, border])
subimage_h, subimage_w = height - 2 * border, width - 2 * border

center_x = np.array([-subimage_w/3, 0, subimage_w/3])
center_y = np.array([-subimage_h/4, subimage_h/4])
X, Y = np.meshgrid(center_x, center_y)
positions = np.vstack([X.ravel(), Y.ravel()]).T
colors = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [9]:
for i, (p, c) in enumerate(zip(positions, colors)):
    vertex_colors = np.zeros((4, 3))
    vertex_colors[2] = c * 0.5
    vertex_colors[3] = c
    vertex_colors[0] = c * 0.5
    draw_vertex_defined_square(p[0], p[1], side, vertex_colors, i + 4)

In [10]:
ps.show()